In [ ]:
import wikipedia
import urllib.request
from pathlib import Path

# 定义要下载的维基百科文章标题
wiki_titles = [
    "San Francisco",
    "Batman",
    "Vincent van Gogh",
    "iPhone",
    "Tesla Model S",
    "BTS band",
]

# 设置数据存储路径
data_path = Path("data_wiki")
if not data_path.exists():
    Path.mkdir(data_path)

# 下载文本数据
for title in wiki_titles:
    page = wikipedia.page(title)
    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(page.content)

# 下载图片数据
image_path = data_path / "images"
if not image_path.exists():
    Path.mkdir(image_path)

image_metadata_dict = {}
image_uuid = 0
MAX_IMAGES_PER_WIKI = 30

for title in wiki_titles:
    page = wikipedia.page(title)
    for i, url in enumerate(page.images):
        if url.endswith(".jpg") or url.endswith(".png"):
            image_uuid += 1
            image_file_name = f"{title}_{image_uuid}.jpg"
            image_metadata_dict[image_uuid] = {
                "filename": image_file_name,
                "img_path": str(image_path / image_file_name),
            }
            urllib.request.urlretrieve(url, image_path / image_file_name)
            if i >= MAX_IMAGES_PER_WIKI:
                break

print(f"Downloaded {len(image_metadata_dict)} images.")

In [ ]:
import os
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.indices import MultiModalVectorStoreIndex
import qdrant_client

# 设置OpenAI API密钥
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 创建Qdrant客户端
client = qdrant_client.QdrantClient(path="qdrant_db")

# 为文本和图像创建单独的向量存储
text_store = QdrantVectorStore(client=client, collection_name="text_collection")
image_store = QdrantVectorStore(client=client, collection_name="image_collection")

# 创建存储上下文
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

# 加载文档
documents = SimpleDirectoryReader("./data_wiki/").load_data()

# 创建多模态索引
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

print("多模态索引创建完成。")

In [ ]:
from llama_index.core.schema import ImageNode
from llama_index.core.response.notebook_utils import display_source_node
import matplotlib.pyplot as plt
from PIL import Image

def plot_images(image_paths):
    plt.figure(figsize=(16, 9))
    for i, img_path in enumerate(image_paths[:6]):
        if os.path.isfile(img_path):
            image = Image.open(img_path)
            plt.subplot(2, 3, i + 1)
            plt.imshow(image)
            plt.axis('off')
    plt.tight_layout()
    plt.show()

def multimodal_query(query, index, top_k=3):
    retriever = index.as_retriever(similarity_top_k=top_k, image_similarity_top_k=5)
    retrieval_results = retriever.retrieve(query)
    
    retrieved_images = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_images.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
    
    plot_images(retrieved_images)

# 测试多模态查询
test_query = "Who are BTS team members and what do they look like?"
multimodal_query(test_query, index)

In [ ]:
test_questions = [
    "Describe the architectural style of San Francisco and show some iconic buildings.",
    "What are the most famous paintings of Vincent van Gogh and how do they look?",
    "Show me the evolution of iPhone designs over the years.",
    "What are the key features of Tesla Model S and how does it look?",
    "Who are the members of BTS and what are their distinct styles?"
]

for question in test_questions:
    print(f"Query: {question}")
    multimodal_query(question, index)
    print("-" * 50)